### Part 2

Estimate the emission parameters from the training set using MLE

In [13]:
import copy
def emission_params_cache(tokens_list: list):
    y_count = {}
    x_given_y_count = {} 
    
    for token in tokens_list:
        if token[1] not in y_count:
            y_count[token[1]] = 1
            x_given_y_count[token[1]] = {}
            x_given_y_count[token[1]][token[0]] = 1
        else:
            y_count[token[1]] += 1
            if token[0] not in x_given_y_count[token[1]]:
                x_given_y_count[token[1]][token[0]] = 1
            else:
                x_given_y_count[token[1]][token[0]] += 1
    
    # calculate emission params
    emission_p = copy.deepcopy(x_given_y_count)
    for label in emission_p:
        for word in emission_p[label]:
            emission_p[label][word] = float(x_given_y_count[label][word]) / y_count[label]
    
    return emission_p
    
def emission_params(tokens_list: list, y: str, x: str, special_token='#UNK#'):
    emission_p = emission_params_cache(tokens_list)
    x_inside = False
    for token in tokens_list:
        if token[0] == x:
            x_inside = True
            break
    
    if not x_inside:
        x = special_token

    if x not in emission_p[y]:
        return 0
    else:
        return emission_p[y][x]
            

In [20]:
# Example ussage
import os

data = [['a', 'O'], ['b', 'O'], ['a', 'I'], ['c', 'O']]
print(emission_params_cache(data))
print(emission_params(tokens_list=data, y='I', x='b'))

# with open('./EN/train') as train_file:
#     read_data = train_file.read()
#     read_data = os.linesep.join([s for s in read_data.splitlines() if s])
#     data = list(map(lambda x: x.split(' '),read_data.split('\n')))
# emission_params(tokens_list=data, y='O', x='a')

{'O': {'a': 0.3333333333333333, 'b': 0.3333333333333333, 'c': 0.3333333333333333}, 'I': {'a': 1.0}}
0


Modify training set to replace words that appear less than k times with special token. Apply this to the emission parameters prediction function with k = 3

In [15]:
def clean_data(tokens_list, k = 1, special_token = '#UNK#'):
    token_freq = {}
    for token in tokens_list:
        if token[0] not in token_freq: 
            token_freq[token[0]] = 1
        else:
            token_freq[token[0]] += 1
    for i in range(len(tokens_list)):
        if token_freq[tokens_list[i][0]] < k:
            tokens_list[i][0] = special_token
    
    return tokens_list

def emission_params_clean_data(tokens_list: list, y: str, x: str):
    tokens_list = clean_data(tokens_list = tokens_list, k = 3)
    return emission_params(tokens_list, y, x)

In [16]:
# Example usage
data = [['a', 'O'], ['b', 'O'], ['a', 'I'], ['c', 'O'], ['a', 'O']]
emission_params_clean_data(tokens_list=data, y='O', x='a')

0.5

Sentiment analysis system that produces the tag for input

In [10]:
def predict_tag(tokens_list: list, x: str):
    y_tags = ['O', 'B-positive', 'I-positive', 'B-neutral', 'I-neutral', 'B-negative', 'I-negative']
    score = 0.0
    y_tag = None
    for y in y_tags:
        y_score = emission_params_clean_data(tokens_list, y, x)
        if (y_score > score):
            y_tag = y
            score = y_score 
    return y_tag

# clean data before predicting the tag
def predict_tag_optimized(tokens_list: list, x: str):
    y_tags = ['O', 'B-positive', 'I-positive', 'B-neutral', 'I-neutral', 'B-negative', 'I-negative']
    score = 0.0
    y_tag = None
    for y in y_tags:
        y_score = emission_params(tokens_list, y, x)
        if (y_score > score):
            y_tag = y
            score = y_score 
    return y_tag
languages = ['EN', 'SG', 'CN', 'FR']

for l in languages:
    with open("./{}/train".format(l)) as train_file:
        read_data = train_file.read()
        read_data = os.linesep.join([s for s in read_data.splitlines() if s])
        data = list(map(lambda x: x.split(' '),read_data.split('\n')))
        data_cleaned = clean_data(tokens_list = data, k = 3)

    with open("./{}/dev.in".format(l)) as in_file, open("./{}/dev.p2.out".format(l), 'w+') as out_file:
        for line in in_file:
            word = line.strip()
            if (word == ''):
                out_file.write("\n")
            else:
                out_file.write("{} {}\n".format(word, predict_tag_optimized(data_cleaned, word)))
    print("Finished: {}".format(l))

Finished: EN
Finished: SG
Finished: CN
Finished: FR


In [12]:
for l in languages:
    output = os.popen("python EvalScript/evalResult.py {0}/dev.out {0}/dev.p2.out".format(l)).read()
    print("Language: {}".format(l))
    print(output)
    print("----------------------")

Language: EN

#Entity in gold data: 226
#Entity in prediction: 1201

#Correct Entity : 165
Entity  precision: 0.1374
Entity  recall: 0.7301
Entity  F: 0.2313

#Correct Sentiment : 71
Sentiment  precision: 0.0591
Sentiment  recall: 0.3142
Sentiment  F: 0.0995

----------------------
Language: SG

#Entity in gold data: 1382
#Entity in prediction: 6599

#Correct Entity : 794
Entity  precision: 0.1203
Entity  recall: 0.5745
Entity  F: 0.1990

#Correct Sentiment : 315
Sentiment  precision: 0.0477
Sentiment  recall: 0.2279
Sentiment  F: 0.0789

----------------------
Language: CN

#Entity in gold data: 362
#Entity in prediction: 3318

#Correct Entity : 183
Entity  precision: 0.0552
Entity  recall: 0.5055
Entity  F: 0.0995

#Correct Sentiment : 57
Sentiment  precision: 0.0172
Sentiment  recall: 0.1575
Sentiment  F: 0.0310

----------------------
Language: FR

#Entity in gold data: 223
#Entity in prediction: 1149

#Correct Entity : 182
Entity  precision: 0.1584
Entity  recall: 0.8161
Entity  F

### Part 3

Estimates the transition parameters from the training set using MLE

In [3]:
import copy
def transition_params(ordered_labels_list: list):
    count = {}
    count_given = {} # 2 layer dictionary depth-0 key is the (i-1)-label, depth-1 key is the i-label
    
    # count frequency of all label and combinations of 2 labels in the dataset
    for idx, label in enumerate(ordered_labels_list):
        if label not in count:
            count[label] = 1
            count_given[label] = {}
            if idx < len(ordered_labels_list) - 1:
                next_label = ordered_labels_list[idx + 1]
                count_given[label][next_label] = 1
        else:
            count[label] += 1
            if idx < len(ordered_labels_list) - 1:
                next_label = ordered_labels_list[idx + 1]
                if next_label not in count_given[label]:
                    count_given[label][next_label] = 1
                else:
                    count_given[label][next_label] += 1
    
    # calculate trans_params
    trans_params = copy.deepcopy(count_given)
    for given_label in trans_params:
        for label in trans_params[given_label]:
            trans_params[given_label][label] /= count[given_label]
            
    return trans_params

def specific_transition_params(ordered_labels_list: list, y: str, y_given: str):
    trans_params = transition_params(ordered_labels_list)
    if y not in trans_params:
        return 0;
    elif y_given not in trans_params[y]:
        return 0;
    else:
        return trans_params[y_given][y]
    
specific_transition_params(['a', 'b', 'b', 'c', 'b', 'a', 'd', 'h', 'b'], 'b', 'a')

0.5

Viterbi algo

In [4]:
def viterbi(sentence: str):
    sentence = sentence.split()
    # sentence has both START and STOP words
    cache = {}
    y_predicted = []
    trans_params = transition_params(sentence)
    print(trans_params)
    for k in range(len(sentence)-1):
         pass
    return result

viterbi("""
Hey I was doing just fine before I met you
Drink too much and that\'s an issue
But I\'m okay
Hey, you tell your friends it was nice to meet them
But I hope I never see them
Again
I know it breaks your heart
Moved to the city in a broke-down car
And four years, no calls
Now you're looking pretty in a hotel bar
And I, I, I, I, I can't stop
No, I, I, I, I, I can't stop
""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



{'Hey': {'I': 1.0}, 'I': {'was': 0.14285714285714285, 'met': 0.14285714285714285, 'hope': 0.14285714285714285, 'never': 0.14285714285714285, 'know': 0.14285714285714285, "can't": 0.2857142857142857}, 'was': {'doing': 0.5, 'nice': 0.5}, 'doing': {'just': 1.0}, 'just': {'fine': 1.0}, 'fine': {'before': 1.0}, 'before': {'I': 1.0}, 'met': {'you': 1.0}, 'you': {'Drink': 0.5, 'tell': 0.5}, 'Drink': {'too': 1.0}, 'too': {'much': 1.0}, 'much': {'and': 1.0}, 'and': {"that's": 1.0}, "that's": {'an': 1.0}, 'an': {'issue': 1.0}, 'issue': {'But': 1.0}, 'But': {"I'm": 0.5, 'I': 0.5}, "I'm": {'okay': 1.0}, 'okay': {'Hey,': 1.0}, 'Hey,': {'you': 1.0}, 'tell': {'your': 1.0}, 'your': {'friends': 0.5, 'heart': 0.5}, 'friends': {'it': 1.0}, 'it': {'was': 0.5, 'breaks': 0.5}, 'nice': {'to': 1.0}, 'to': {'meet': 0.5, 'the': 0.5}, 'meet': {'them': 1.0}, 'them': {'But': 0.5, 'Again': 0.5}, 'hope': {'I': 1.0}, 'never': {'see': 1.0}, 'see': {'them': 1.0}, 'Again': {'I': 1.0}, 'know': {'it': 1.0}, 'breaks': {'yo

NameError: name 'result' is not defined